# Elementary Sorts

The **sorting problem** is to re-arrange an array of $n$ items into ascending order according to a defined key which is part of the item. The goal is to sort any type of data - but how to know how to sort different types of data without knowing the type of the item's key?

A **callback** is a reference to executable code. The client passes the array of objects to the `sort()` function, and the `sort()` function calls back object's `compareTo()` method as needed. In Python, you implement a callback with a first-class function.

There's a compare function (Java has the Comparable interface) that handles the different data types, and defines how to determine order between two items. The **total order** is a binary relation $\leq$ that satisfies:

- Antisymmetry: if $v \leq w$ and $w \leq v$, then $v = w$
- Transitivity: if $v \leq w$ and $w \leq x$, then $v \leq x$
- Totality: either $v \leq w$ or $w \leq v$ or both

Examples are alphabetical order for strings or numerical ordering of numbers. Rock, paper, scissors would not work under transitivity.

When you implement a `compare()` function:
- There is a total order
- It (`v.compareTo(w)`) returns a negative integer, zero, or positive integer when $v$ is less than, equal to, or greater than $w$, respectively
- It throws an exception if they are incomparable types (or either is `null`)

Two useful abstractions (helper functions) include one for checking if an item is less than another (uses the `compareTo()` function), and one to exchange the item at index `i` with that at index `j`.

## Selection Sort

Start with a group of items out of order, in iteration `i`, you find the index of the smallest remaining entry and swap `a[i]` with `a[min]`.

The algorithm has a pointer (or sliding index) that scans from left to right. Invariants: entries to the left of the pointer (including the pointer) are fixed and in ascending order, and no entry to the right of the pointer is smaller than any entry to the left of it. The algorithm maintains the invariant.

**Mathematical analysis**: selection sort uses $(N - 1) + (N - 2) + \ldots + 1 + 0 \approx \frac{N^2}{2}$ compares and $N$ exchanges.

**Running time** is insensitive to input: quadratic time, even if the input is sorted.

**Data movement is minimal**: there's a linear number of exchanges.

In [23]:
def selectionSort(arr):
    for i in range(len(arr)):
        min_item = min(arr[i:])
        min_index = arr[i:].index(min_item)
        arr[i + min_index], arr[i] = arr[i], min_item
    return arr

a = [7, 5, 4, 3, 1, 6, 8, 2, 10, 9]
print("Array to be sorted: {}".format(a))
print("Selection sort final items: {}".format(selectionSort1(a)))

Array to be sorted: [7, 5, 4, 3, 1, 6, 8, 2, 10, 9]
Selection sort final items: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Insertion Sort

Start with a group of items out of order, in iteration `i`, swap `a[i]` with each larger entry to its left.

The algorithm has a pointer (or sliding index) that scans from left to right. Invariants: entries to the left of the pointer (including it) are in ascending order, and entries to the right of the pointer have not yet been seen.

**Mathematical analysis**: to sort a randomly-ordered array with distinct keys, insertion sort uses $~ \frac{1}{4} N^2$ compares and $~ \frac{1}{4} N^2$ exchanges on average.

**Best case**: if the array is in ascending order, insertion sort makes $N-1$ compares and 0 exchanges. (Linear vs. quadratic in selection sort).

**Worst case**: if the array is in descending order (and there are no duplicates), insertion sort makes $~ \frac{1}{2} N^2$ compares and $~ \frac{1}{2} N^2$ exchanges. (Slower than selection sort - same compares but more exchanges).

There is a use-case where insertion sort runs on linear time:

An **inversion** is a pair of keys that are out of order. An array is **partially sorted** if the number of inversions is $\leq c N$. One example is a subarray of size 10 appended to a sorted subarray of size $N$. Another is an array of size $N$ with only 10 entries out of place.

The proposition is that insertion sort runs on linear time for partially sorted arrays - the number of exchanges equals the number of inversions (number of compares = exchanges + ($N - 1$)).

In [24]:
def insertionSort(arr):
    for i in range(len(arr)):
        for j in range(i, 0, -1):
            if arr[j] < arr[j - 1]:
                arr[j], arr[j - 1] = arr[j - 1], arr[j]
            else:
                break
    return arr

b = [3, 7, 9, 1, 4, 5, 10, 6, 8, 2]
print("Array to be sorted: {}".format(b))
print("Insertion sort final items: {}".format(insertionSort(b)))

Array to be sorted: [3, 7, 9, 1, 4, 5, 10, 6, 8, 2]
Insertion sort final items: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Shellsort

Shellsort came about because insertion sort is inefficient (elements only move one position at a time, even when they have a long way to go to be in order). Shellsort uses *h-sorting* to move entries more than one position at a time.

**H-sorting** looks at the array in h-length subsequences, so an h-sorted array is h "interleaved sorted subsequences" (where h is some number, and you look at every hth item, then those items are sorted). Look at decreasing values of h to sort the full array.

Each sort is implemented with only a few exchanges, assuming the relevant higher h-sorts were performed in the round prior. It's like insertion sort, but with a stride of length h instead of length 1. For example, perform a 7-sort, then 3-sort, finally a 1-sort.

The proposition for shellsort is that a $g$-sorted array remains $g$-sorted after $h$-sorting it. The mathematical proof is subtle, but shellsort is efficient because of this fact.

The big question is, what increment sequence for h do you use?

- Powers of 2 (1, 2, 4, 8, 16, 32, ...): No
- Powers of 2 minus one (1, 3, 7, 15, 31, 63, ...): Maybe
- $3x + 1$ (1, 4, 13, 40, 121, 364, ...): Okay (easy to compute)
- Sedgewick merging of $(9 \times 4^i) - (9 \times 2^i) + 1$ and $4^i - (3 \times 2^i) + 1$ to get (1, 5, 19, 41, 109, 209, 505, 929, ...): Good (tough to beat in empirical studies)

The proposition is that shellsort's worst case number of compares with the $3x+1$ increments is $O (N^{\frac{3}{2}})$ (analysis is still open). In practice, the number of compares is less than that - no one has found an accurate model.

Shellsort is an example of a simple idea that led to substantial performance gains, and is useful in practice. It's fast, unless the array size is huge, it has a tiny, fixed footprint for code or hardware sort prototypes.

It's a simple algorithm with nontrivial performance and raises interesting questions. Does it have an asymptotic growth rate? What is the best sequence of increments? What is the average-case performance?

**Best case**: if the array is already sorted, shellsort (using $3x+1$ increment sequence) makes a linearithmic number of compares. Each successive increment value of h differs by at least a factor of 3, so there are ~$\log_3 n$ increment values. For each increment value h, the array is already h-sorted, so it will make ~$n$ compares.

In [25]:
def shellSort(arr):
    # Uses Knuth's 3x+1 increment
    N = len(arr)
    h = 1
    while (h < N // 3):
        h = 3 * h + 1
        
    while (h >= 1):
        # h-sort the array
        for i in range(h, N):
            for j in range(i, h - 1, -h):
                if arr[j] < arr[j - h]:
                    arr[j], arr[j - h] = arr[j - h], arr[j]
        h = h // 3
    
    return arr

c = [5, 6, 3, 9, 1, 7, 10, 8, 2, 4]
print("Array to be sorted: {}".format(c))
print("Shellsort final items: {}".format(insertionSort(c)))

Array to be sorted: [5, 6, 3, 9, 1, 7, 10, 8, 2, 4]
Shellsort final items: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


## Shuffling

Shuffling is rearranging a set of items into a random order. One way to do this is to use a sort - generate a random real number for each array entry, then sort using that number as the key. Is there a faster way to shuffle (without the overhead of a full sort)?

There is a way to rearrange an array so the result is a uniformly random permutation in linear time. In iteration `i`, pick an integer `r` between 0 and `i` uniformly at random, then swap `a[i]` and `a[r]`.

A common bug is choosing a random integer r between 0 and $N-1$ each pass - doesn't work. An alternative variant that is correct is to choose r between `i` and $N-1$ (what you haven't seen).

In [26]:
def knuthShuffle(arr):
    import random
    for i in range(len(arr)):
        r = random.randint(0, i)
        arr[r], arr[i] = arr[i], arr[r]
    return arr

d = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
print("Array to be shuffled: {}".format(d))
print("Knuth shuffle final items: {}".format(knuthShuffle(d)))

Array to be shuffled: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Knuth shuffle final items: [10, 8, 4, 3, 7, 2, 1, 5, 6, 9]


## Convex Hull

A **convex hull** of a set of $N$ points is the smallest perimeter fence enclosing the points. Equivalent definitions say it's the smallest convex set containing all the points, it's the smallest area convex polygon enclosing the points, or it's a convex polygon enclosing the points, whose vertices are points in set.

Given a set of points, you want a program that outputs the convex hull - a sequence of vertices in counterclockwise order. Points that fall on the boundary but aren't vertices shouldn't be included.

Some applications:

- A robot wants to maneuver around an obstacle defined by some polygon - shortest path will be along the convex hull
- Given $N$ points in the plane, find a pair of points with the largest Euclidean distance between them (this pair will be extreme points on the convex hull)

Some geometric properties:

- Can traverse the convex hull by making only counterclockwise turns
- The vertices of the convex hull appear in increasing order of polar angle with respect to point $p$ with the lowest y-coordinate

The **Graham scan** algorithm relies on the two facts above:

- Choose point $p$ with the smallest y-coordinate
- Sort the points by polar angle with $p$
- Consider points in order; discard unless it creates a counterclockwise turn


Some questions for the Graham scan include how do you find the point with the smallest y-coordinate? Do a sort. You also need to sort the points by the smallest polar angle of the start point to that point vs. the x-axis, so the bulk of the work for this algorithm is in the sorting. Next issue is given three points, is a turn $a \to b \to c$ a counterclockwise turn? You need to watch for other issues like edge cases (infinity slopes and collinearity of points), and floating point number precision.